# Boilerplate

Packae installation, loading, and dataloaders. There's also a simple model defined. You can change it your favourite architecture if you want.

In [60]:
!pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [68]:
# The last argument 'targeted' can be used to toggle between a targeted and untargeted attack.
def fgsm(model, x, target, eps, targeted=True):
    ###############################################
    # TODO fill me
    adv_x = []
    if targeted:
        x.requires_grad_() # this is required so we can compute the gradient w.r.t x
        # compute gradient
        # note that CrossEntropyLoss() combines the cross-entropy loss and an implicit softmax function
        L = nn.CrossEntropyLoss()
        loss = L(model(x), target) # TO LEARN: make sure you understand this line
        loss.backward()
        adv_x = x - eps * torch.sign(x.grad)
        new_class = model(adv_x).argmax(dim=1).item()
        assert(new_class == target)
    else:
        x.requires_grad_() # this is required so we can compute the gradient w.r.t x
        # compute gradient
        # note that CrossEntropyLoss() combines the cross-entropy loss and an implicit softmax function
        L = nn.CrossEntropyLoss()
        loss = L(model(x), target) # TO LEARN: make sure you understand this line
        loss.backward()
        adv_x = x + eps * torch.sign(x.grad)
    ###############################################
    return adv_x


def pgd_untargeted(model, x, labels, k, eps, eps_step):
    ###############################################
    # TODO fill me
    # print(x)
    alpha = eps_step
    adv_x = x.clone().detach().requires_grad_(True).to(device)
    for i in range(k):
        L = nn.CrossEntropyLoss()
        loss = L(model(adv_x), labels) # TO LEARN: make sure you understand this line
        loss.backward()
        # print(adv_x)
        pertu = alpha * torch.sign(adv_x.grad)
        adv_x = torch.clip(adv_x + pertu, min=x-eps, max=x+eps)
        adv_x = adv_x.detach().clone()
        adv_x.requires_grad_() 
    ###############################################
    return adv_x

# Implement Adversarial Training

In [ ]:
def train_model(model, num_epochs, enable_defense=True):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):

            if enable_defense:
                pass
                ###############################################
                # Fill code here to do adversarial training
                # You may find it useful to switch to 'eval' model while generating the attack
                # and switch back again to 'train' mode once the attack is generated.
                ###############################################

            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc = 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))

# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.